<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Semantic_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Sample data
data = ["The movie is awesome. It was a good thriller",
        "We are learning NLP throughg GeeksforGeeks",
        "The baby learned to walk in the 5th month itself"]

# Tokenizing the data
tokenized_data = [word_tokenize(document.lower()) for document in data]

# Creating TaggedDocument objects
tagged_data = [TaggedDocument(words=words, tags=[str(idx)]) for idx, words in enumerate(tokenized_data)]


# Training the Doc2Vec model
model = Doc2Vec(vector_size=100, window=2, min_count=1, workers=4, epochs=1000)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Infer vector for a new document
new_document = "The baby was laughing and palying"
inferred_vector = model.infer_vector(word_tokenize(new_document.lower()))

# Find most similar documents
similar_documents = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Print the most similar documents
for index, score in similar_documents:
    print(f"Document {index}: Similarity Score: {score}")
    print(f"Document Text: {data[int(index)]}")
    print()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Document 2: Similarity Score: 0.9834631085395813
Document Text: The baby learned to walk in the 5th month itself

Document 0: Similarity Score: 0.9460567831993103
Document Text: The movie is awesome. It was a good thriller

Document 1: Similarity Score: 0.8827896118164062
Document Text: We are learning NLP throughg GeeksforGeeks



In [18]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4d6677ba28cfb95e26b75028a63715d0ae0a89a96ba84ae8f59efa0c1f19cb8d
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [21]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample sentence
sentences = ["The movie is awesome. It was a good thriller",
        "We are learning NLP throughg GeeksforGeeks",
        "The baby learned to walk in the 5th month itself"]



In [63]:
from scipy.spatial import distance


test = "I liked the movie"
test_vec = model.encode([test])[0]


for sent in sentences:
#  similarity_score= dot(test_vec, model.encode([sent])[0])/(norm(test_vec)*model.encode([sent])[0])
  similarity_score = distance.cosine(test_vec,model.encode([sent])[0])
  print(f'similarity = {similarity_score} for {sent}')

similarity = 0.3368104100227356 for The movie is awesome. It was a good thriller
similarity = 0.8930581361055374 for We are learning NLP throughg GeeksforGeeks
similarity = 0.9568249434232712 for The baby learned to walk in the 5th month itself


In [56]:
from numpy import dot
from numpy.linalg import norm
a = np.array([1.0,2.0,3.0,4.0])
b= np.array([3.0,4.0,5.0,6.0])
# cos_sim = dot(a, b)/(norm(a)*norm(b))
# cos_sim

In [57]:
#np.cos(a,b)

ImportError: cannot import name 'cosine' from 'scipy' (/usr/local/lib/python3.10/dist-packages/scipy/__init__.py)

In [61]:
from scipy.spatial import distance
distance.cosine(test_vec,test_vec)

0

In [50]:
distance.cosine(np.array([1, 0, 0]), np.array([1, 0, 0]))

0

In [49]:
type(test_vec)

numpy.ndarray

In [54]:
a

array([1., 2., 3., 4.])

In [55]:
b

array([ 0.54030231, -0.41614684, -0.9899925 , -0.65364362])